###데이터 다운로드


In [ ]:
import pandas as pd

df = pd.read_excel("/content/b_ind.xlsx")

###집단 만들기

In [ ]:
#결측치
df['I_INC1'] = df['I_INC1'].fillna(0)
df['I_INC2'] = df['I_INC2'].fillna(0)
df['EROOP'] = df['EROOP'].fillna(0)
df['INOOP'] = df['INOOP'].fillna(0)
df['OUOOP_1'] = df['OUOOP_1'].fillna(0)
df['OUOOP_2'] = df['OUOOP_2'].fillna(0)
df.dropna(subset=['CD', 'CD1_1', 'HS2_YN', 'HS1', 'HT', 'WT'], inplace=True)

#필요한 칼럼만 남기기
df = df[['SEX', 'BIRTH_Y', 'MARR', 'EDU', 'DISA_YN', 'HEALTH_INS', 'I_PHI1_YN',
         'ECO1', 'CD1_1', 'HS2_YN', 'HS1', 'HT', 'WT', 'INGUN', 'P1', 'S3',
         'D1', 'CD', 'EROOP', 'INOOP', 'OUOOP_1', 'OUOOP_2', 'I_INC1', 'I_INC2']]

#사용할 표본집단
df2020_ind = df[
    (df['CD'] != -9) &
    (df['CD1_1'] != -9) &
    (df['I_INC2'] != -9) &
    ((df['I_INC1'] + df['I_INC2']) > 0) &
    (df['EROOP'] != -9) &
    (df['INOOP'] != -9) &
    (df['OUOOP_1'] != -9) &
    (df['OUOOP_2'] != -9) &
    (df['BIRTH_Y'] >= 1960) &
    (df['BIRTH_Y'] <= 2001) &
    (df['S3'] != -9) &
    (df['D1'] != -9) &
    (df['I_PHI1_YN'] != 3)
]

###처치/결과 변수 정의 및 파생변수 생성

In [ ]:
df['AGE'] = year - df['BIRTH_Y']+1  # 나이
df['INCOME'] = df['I_INC1'] + df['I_INC2']  # 수입
df['MED_EXP'] = ((
    df['EROOP'] +
    df['INOOP']+
    df['OUOOP_1']+
    df['OUOOP_2'])/10000
)  # 의료비

df['overburden_yn'] = (df['MED_EXP'] / df['INCOME'] >= 0.1).astype(int) #의료비과부담 유무
df['CD_total'] = ((df['CD'] == 1) | (df['CD1_1'] == 1)).astype(int) #만성질환 유무

#변수 생성 후 필요없는 칼럼 없애기
df.drop(columns=['HT', 'WT', 'BIRTH_Y', 'I_INC1', 'I_INC2', 'EROOP', 'INOOP', 'OUOOP_1', 'OUOOP_2', 'MED_EXP', 'INCOME', 'CD'], inplace=True)

###공변량 변환

In [ ]:
#명목형변수
# SEX: 남성(1) -> 1, 여성(2) -> 0
df['SEX'] = df['SEX'].replace({1: 1, 2: 0})

# DISA_YN : 장애 있음(1) -> 1, 장애 없음(2) -> 0
df['DISA_YN'] = df['DISA_YN'].replace({1: 1, 2: 0})

# HS2_YN : 우울감 있음(1) -> 1, 나머지 -> 0
df['HS2_YN'] = df['HS2_YN'].apply(lambda x: 1 if x == 1 else 0)

# P1 : 운동을 규칙적으로 함(1) -> 1, 나머지 -> 0
df['P1'] = df['P1'].apply(lambda x: 1 if x == 1 else 0)

# MARR: 배우자 있음(1) -> 1, 없음(2) -> 0
df['MARR'] = df['MARR'].apply(lambda x: 1 if x in [1, 2] else 0)

# S3: 2이하 -> 1, 나머지 -> 1
df['S3'] = df['S3'].apply(lambda x: 1 if x in [1, 2] else 0)

# EDU: 3이하 -> 0, 4,5 -> 1
df['EDU'] = df['EDU'].apply(lambda x: 1 if x in [4, 5] else 0)

# HEALTH_INS: 원핫 인코딩
df = pd.get_dummies(df, columns=['HEALTH_INS'], prefix='HEALTH_INS', drop_first=True, dtype=int)
#첫 번째 카테고리를 제외하여 다중공선성을 방지합니다.

# I_PHI1_YN : 민간의료보험 가입했음 -> 1, 나머지 -> 0
df['I_PHI1_YN'] = df['I_PHI1_YN'].apply(lambda x: 1 if x == 1 else 0)

# ECO1: 4이하 -> 1, 5,6 -> 1
df['ECO1'] = df['ECO1'].apply(lambda x: 1 if x in [1, 2, 3, 4] else 0)

#순서형변수
# HS1 (스트레스 인지정도): 4(없다) -> 0, 나머지(1,2,3) -> 1
df['HS1'] = df['HS1'].apply(lambda x: 0 if x == 4 else 1)

# D1 (음주 빈도): 1-4(거의 안 함) -> 0, 나머지(5,6,7,8) -> 1
df['D1'] = df['D1'].apply(lambda x: 0 if x in [1, 2, 3, 4] else 1)

###정제된 데이터 다운로드

In [ ]:
df.to_excel("final_dataset.xlsx", index=False)